<a href="https://colab.research.google.com/github/AbhiRishi96/qure_ai_problems/blob/main/qure_ai_problem_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/AbhiRishi96/qure_ai_problems.git

Cloning into 'qure_ai_problems'...
remote: Enumerating objects: 208, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 208 (delta 50), reused 93 (delta 43), pack-reused 82
Receiving objects: 100% (208/208), 200.96 MiB | 25.71 MiB/s, done.
Resolving deltas: 100% (65/65), done.
Checking out files: 100% (125/125), done.


## INSTALL AND IMPORT REQUIRED LIBRARIES

In [ ]:
!pip install PyMuPDF Pillow
!pip install easyocr

In [3]:
import fitz
import io
import pandas as pd
from PIL import Image
import PIL
from PIL import ImageDraw
import easyocr

In [4]:
reader = easyocr.Reader(['en'], gpu = True)

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

## Extracting image from pdf

In [5]:
file = "/content/qure_ai_problems/Problem_3/input/Data_Eng_assignment.pdf"

In [7]:
pdf_file = fitz.open(file)
page_index = 2    # image is present at only this page in the document
page = pdf_file[page_index]
image_li = page.get_images()
#printing number of images found in this page
#page index starts from 0 hence adding 1 to its content
if image_li:
    print(f"[+] Found a total of {len(image_li)} images in page {page_index+1}")
else:
    print(f"[!] No images found on page {page_index+1}")
for image_index, img in enumerate(page.get_images(), start=1):
    #get the XREF of the image
    xref = img[0]
    #extract the image bytes
    base_image = pdf_file.extract_image(xref)
    image_bytes = base_image["image"]
    #get the image extension
    image_ext = base_image["ext"]
    #load it to PIL
    image = Image.open(io.BytesIO(image_bytes))
    #save it to local disk
    image.save(open(f"/content/qure_ai_problems/Problem_3/output/" + "extracted_image.png", "wb"))

[+] Found a total of 1 images in page 3


## EXTRACTING TEXT FROM IMAGE

In [8]:
path_to_image = '/content/qure_ai_problems/Problem_3/output/extracted_image.png'

In [9]:
### EXTRACT AND STORE TEXT INTO PANDAS DATAFRAME
extract_text = reader.readtext(path_to_image, detail = 0)
df_extracted_text = pd.DataFrame(extract_text)

#### DATA CLEANING

In [10]:
### TO KEEP ONLY REQUIRED VALUES (i.e. DATA PRESENT IN RED BOX IN THE IMAGE)
df_extracted_text = df_extracted_text.iloc[27:52]

In [11]:
### RENAMING AND DELETING WRONG INTERPRETED DATA
df_extracted_text = df_extracted_text.replace({'#ITaxable': '#/Taxable',
                                               'Test Product _': 'Test Product 2 (Service)',
                                               '(Service': '1',
                                               'P1ooo': 'P1000',
                                               'Test Product': 'Test Product 1'})
df_extracted_text = df_extracted_text.drop([45])

In [12]:
### ADDING BACK MISSING DATA FROM THE IMAGE
df_extracted_text.loc[40.5] = '1'
df_extracted_text.loc[44.5] = '1'
df_extracted_text.loc[49.5] = '1'
df_extracted_text = df_extracted_text.sort_index().reset_index(drop = True)

In [13]:
### SAVING EXTRACTED DATA FROM IMAGE INTO EXCEL FILE
df_extracted_text.to_excel(f'/content/qure_ai_problems/Problem_3/output/' + 'text_from_image.xlsx', index = False, header = False)

In [14]:
df_extracted_text

,0
0,PO #
1,Sales Rep: Name
2,Ship Date
3,Ship Via
4,Terms
5,Due Date
6,3/18/2015
7,#/Taxable
8,Description
9,Quantity
